<a href="https://colab.research.google.com/github/mojo46/Machine-Learning-noob/blob/master/TextRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 2.99 MiB | 2.38 MiB/s, done.
Resolving deltas: 100% (239/239), done.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
cwd = os.getcwd()
def unzip_a_zip(file_path, dest_dir=cwd):
  from zipfile import ZipFile
  with ZipFile(file_path, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(dest_dir)

In [0]:
unzip_a_zip("/content/clear_alpha_num_output.zip") # enter zip file path and destination path if nessary

In [0]:
!unzip  '/content/drive/My Drive/DataSet/outputNP3.zip' -d '/content'

In [0]:
model_path='/content/drive/My Drive/pretrained_model/TPS-ResNet-BiLSTM-Attn.pth'

image_folder = '/content/output'

In [0]:
!ls '/content/drive/My Drive/pretrained_model/TPS-ResNet-BiLSTM-Attn.pth'

'/content/drive/My Drive/pretrained_model/TPS-ResNet-BiLSTM-Attn.pth'


In [0]:
#write dataframe to a csv file
def write_to_csv(file_name,df):
  import csv
  with open(f'{df}', 'w') as csvfile:
      writer=csv.writer(csvfile, delimiter=',')
      writer.writerows(df)

In [0]:
!python '/content/deep-text-recognition-benchmark/demo.py' \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder '/content/output' \
--saved_model '/content/drive/My Drive/pretrained_model/TPS-ResNet-BiLSTM-Attn.pth'

In [0]:
!cat /content/deep-text-recognition-benchmark/demo.py

In [0]:
%%writefile /content/deep-text-recognition-benchmark/demo.py

import string
import argparse
import csv
import torch
import torch.backends.cudnn as cudnn
import torch.utils.data
import torch.nn.functional as F

from utils import CTCLabelConverter, AttnLabelConverter
from dataset import RawDataset, AlignCollate
from model import Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def demo(opt):
    """ model configuration """
    if 'CTC' in opt.Prediction:
        converter = CTCLabelConverter(opt.character)
    else:
        converter = AttnLabelConverter(opt.character)
    opt.num_class = len(converter.character)

    if opt.rgb:
        opt.input_channel = 3
    model = Model(opt)
    print('model input parameters', opt.imgH, opt.imgW, opt.num_fiducial, opt.input_channel, opt.output_channel,
          opt.hidden_size, opt.num_class, opt.batch_max_length, opt.Transformation, opt.FeatureExtraction,
          opt.SequenceModeling, opt.Prediction)
    model = torch.nn.DataParallel(model).to(device)

    # load model
    print('loading pretrained model from %s' % opt.saved_model)
    model.load_state_dict(torch.load(opt.saved_model, map_location=device))

    # prepare data. two demo images from https://github.com/bgshih/crnn#run-demo
    AlignCollate_demo = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)
    demo_data = RawDataset(root=opt.image_folder, opt=opt)  # use RawDataset
    demo_loader = torch.utils.data.DataLoader(
        demo_data, batch_size=opt.batch_size,
        shuffle=False,
        num_workers=int(opt.workers),
        collate_fn=AlignCollate_demo, pin_memory=True)

    # predict
    model.eval()
    with torch.no_grad():
        with open('results.csv', 'w') as csvfile:
          writer=csv.writer(csvfile, delimiter=',')

          for image_tensors, image_path_list in demo_loader:
              batch_size = image_tensors.size(0)
              image = image_tensors.to(device)
              # For max length prediction
              length_for_pred = torch.IntTensor([opt.batch_max_length] * batch_size).to(device)
              text_for_pred = torch.LongTensor(batch_size, opt.batch_max_length + 1).fill_(0).to(device)

              if 'CTC' in opt.Prediction:
                  preds = model(image, text_for_pred).log_softmax(2)

                  # Select max probabilty (greedy decoding) then decode index to character
                  preds_size = torch.IntTensor([preds.size(1)] * batch_size)
                  _, preds_index = preds.max(2)
                  preds_index = preds_index.view(-1)
                  preds_str = converter.decode(preds_index.data, preds_size.data)

              else:
                  preds = model(image, text_for_pred, is_train=False)

                  # select max probabilty (greedy decoding) then decode index to character
                  _, preds_index = preds.max(2)
                  preds_str = converter.decode(preds_index, length_for_pred)

              print('-' * 80)
              print(f'{"image_path":25s}\t{"predicted_labels":25s}\tconfidence score')
              print('-' * 80)
              preds_prob = F.softmax(preds, dim=2)
              preds_max_prob, _ = preds_prob.max(dim=2)
              print('image',image_path_list,'predi   ',preds_str)

              for img_name, pred, pred_max_prob in zip(image_path_list, preds_str, preds_max_prob):
                  if 'Attn' in opt.Prediction:
                      pred_EOS = pred.find('[s]')
                      pred = pred[:pred_EOS]  # prune after "end of sentence" token ([s])
                      pred_max_prob = pred_max_prob[:pred_EOS]

                  # calculate confidence score (= multiply of pred_max_prob)
                  confidence_score = pred_max_prob.cumprod(dim=0)[-1]

                  # print(f'{img_name}\t{pred}\t{confidence_score:0.4f}')
                  # print(f'{img_name:25s}\t{pred:25s}\t{confidence_score:0.4f}')
                  print(f'{img_name}\t{pred}\t{confidence_score}')
                  confi = f'{confidence_score:0.4f}'
                  results1 = [(img_name,pred,confi)]
                  writer.writerows(results1)
                    

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--image_folder', required=True, help='path to image_folder which contains text images')
    parser.add_argument('--workers', type=int, help='number of data loading workers', default=4)
    parser.add_argument('--batch_size', type=int, default=192, help='input batch size')
    parser.add_argument('--saved_model', required=True, help="path to saved_model to evaluation")
    """ Data processing """
    parser.add_argument('--batch_max_length', type=int, default=25, help='maximum-label-length')
    parser.add_argument('--imgH', type=int, default=32, help='the height of the input image')
    parser.add_argument('--imgW', type=int, default=100, help='the width of the input image')
    parser.add_argument('--rgb', action='store_true', help='use rgb input')
    parser.add_argument('--character', type=str, default='0123456789abcdefghijklmnopqrstuvwxyz', help='character label')
    parser.add_argument('--sensitive', action='store_true', help='for sensitive character mode')
    parser.add_argument('--PAD', action='store_true', help='whether to keep ratio then pad for image resize')
    """ Model Architecture """
    parser.add_argument('--Transformation', type=str, required=True, help='Transformation stage. None|TPS')
    parser.add_argument('--FeatureExtraction', type=str, required=True, help='FeatureExtraction stage. VGG|RCNN|ResNet')
    parser.add_argument('--SequenceModeling', type=str, required=True, help='SequenceModeling stage. None|BiLSTM')
    parser.add_argument('--Prediction', type=str, required=True, help='Prediction stage. CTC|Attn')
    parser.add_argument('--num_fiducial', type=int, default=20, help='number of fiducial points of TPS-STN')
    parser.add_argument('--input_channel', type=int, default=1, help='the number of input channel of Feature extractor')
    parser.add_argument('--output_channel', type=int, default=512,
                        help='the number of output channel of Feature extractor')
    parser.add_argument('--hidden_size', type=int, default=256, help='the size of the LSTM hidden state')

    opt = parser.parse_args()

    """ vocab / character number configuration """
    if opt.sensitive:
        opt.character = string.printable[:-6]  # same with ASTER setting (use 94 char).

    cudnn.benchmark = True
    cudnn.deterministic = True
    opt.num_gpu = torch.cuda.device_count()

    demo(opt)

Overwriting /content/deep-text-recognition-benchmark/demo.py


Commands to Edit code in colabs itself

In [0]:
%pycat /content/deep-text-recognition-benchmark/demo.py

In [0]:
!rm /content/deep-text-recognition-benchmark/demo.py

In [0]:
%%writefile /content/deep-text-recognition-benchmark/demo.py

Filter data by catagory

In [0]:
import pandas as pd
import numpy as np
import os
import shutil

In [0]:
#Import csv to Dataframe and convert class column lower case to upper
resultsss =  pd.read_csv("/content/results.csv",header=None)
resultsss[1] = resultsss[1].apply(lambda x: x.upper())
resultsss

,0,1,2
0,/content/output/0.jpg,2,0.9995
1,/content/output/1.jpg,0,0.3818
2,/content/output/2.jpg,1,0.6789
3,/content/output/3.jpg,2,0.9995
4,/content/output/4.jpg,A,0.9998
...,...,...,...
9922,/content/output/9922.jpg,6,0.9995
9923,/content/output/9923.jpg,V,0.9717
9924,/content/output/9924.jpg,1,0.9969
9925,/content/output/9925.jpg,4,0.9998


Function to generate alphabets and digits list of char

In [0]:
# Function to generate alphabets and digits list of charectors
def alpha_num_list():
  alphabets = [] 
  alpha = 'A'
  for i in range(0, 26): 
      alphabets.append(alpha) 
      alpha = chr(ord(alpha) + 1)
  numbers =  list(range(0,10))
  AplhaNum_list = numbers + alphabets

  return AplhaNum_list

In [0]:
apha_num_list1 = list(map(str,alpha_num_list()))
apha_num_list1

In [0]:
resultsss[ resultsss[1]== "I"]

Test number of occurence and not in dataframe of elements in list (eg: aplhabets and numbers list)

In [0]:
def num_of_occurence_in_df(df,list):
  count = 0
  for i in list:
    shape1 = df[df[1] == i].shape
    print(f'count of {i} is {shape1[0]}')
    if (shape1[0]== 0 ):
      print("alhabet is not present >> ",i)

In [0]:
num_of_occurence_in_df(resultsss,apha_num_list)

In [0]:
alpha_and_nums_df = resultsss[(resultsss[2]>0.75) &(resultsss[2]>0) & (resultsss[1].isin(apha_num_list1)) ]
print(alpha_and_nums_df)

In [0]:
os.chdir("/content")
os.mkdir("/content/clear_alpha_num_output")
for i in alpha_num_list():
  os.mkdir(f'/content/clear_alpha_num_output/{i}')

In [0]:
!ls /content/clear_alpha_num_output

In [0]:
np_alph_num = np.asarray(alpha_and_nums_df)

In [0]:
np_alph_num.shape

(5826, 3)

In [0]:
count=0
for a in np_alph_num:
  if(a[1] in (apha_num_list1)):
    count+=1
    shutil.copy(a[0],f'/content/clear_alpha_num_output/{a[1]}')
    print(f'Copy {a[0]} to /content/clear_alpha_num_output/{a[1]} : {count}')
    
print(count)

In [0]:
!ls -l -R /content/sorted | wc -l

5972


In [0]:
!zip -r  "/content/sorted.zip" "/content/sorted"

In [0]:
!cp -r '/content/clear_alpha_num_output' '/content/sorted'

In [0]:
df_75 = pd.DataFrame({'0': np_alph_num[:, 0], '1': np_alph_num[:, 1],'2': np_alph_num[:, 2]})
df_75

,0,1,2
0,/content/output/0.jpg,2,0.9995
1,/content/output/3.jpg,2,0.9995
2,/content/output/4.jpg,A,0.9998
3,/content/output/7.jpg,2,0.999
4,/content/output/9.jpg,1,0.9069
...,...,...,...
5821,/content/output/9921.jpg,2,0.9998
5822,/content/output/9922.jpg,6,0.9995
5823,/content/output/9923.jpg,V,0.9717
5824,/content/output/9924.jpg,1,0.9969


In [0]:
type(df_75)

pandas.core.frame.DataFrame

In [0]:
df_75 = resultsss[(resultsss[2]>0.75) &(resultsss[2]>0) & (resultsss[1].isin(apha_num_list1)) ]

In [0]:
num_of_occurence_in_df(df_75,apha_num_list)